# Import Widget Libraries

In [1]:
%load_ext autoreload   
%autoreload 2
import kyurem
from kyurem.core.idom_loader import reload_bundle
from kyurem.core.widget import WidgetModel

kyurem_client: 0.0.0


In [68]:
from kyurem import Service
service = Service(kh = 'indeed.v0.0.1:612e3403')

## Load data 

In [69]:
import pandas as pd
df1 = pd.read_csv("indeed.v0.0.1_612e3403_sampled.csv")
corpus_data = df1.to_dict('records')

df2 = pd.read_csv("indeed.v0.0.1_612e3403_merged.csv")
merge_data = df2.to_dict('records')

In [70]:
# TODO: pass column na
service.load_corpus(corpus_data, 'parent_title', 'context', 'child_title')

In [71]:
service.load_merge_data(merge_data, 'corpus_entity', 'node_label', 'node_uuid', 'node_title')

In [72]:
decision_list = ["Accept", "Reject", "Defer"]

In [73]:
mergedata = service.get_merge_data()["rows"]

# Create Widget

In [79]:
from kyurem import MergeVerifier


def init(state): 
    # Fetch initial view data
    return {
        "subgraph": None,
        "corpus": None,
        "mergedata":mergedata,
        "decisions":decision_list,
    }

def focus(state, row, panel):
    # node can be in the following format:
    # node : { "node_label": str, "node_property": str, "node_property_value": str }
    node = { "node_label": row["node_label"], "node_property": "title", "node_property_value": row["node"] }
    entity = { "node_label": row["node_label"], "node_property": "title", "node_property_value": row["entity"] }
    # Fetch the neighborhood around the input node 
    result = service.get_node_neighborhood(node)
    datatable = service.get_annotated_corpus(entity)
    
    data = {}
    data["subgraph"] = result["schema"]
    # TODO: Move conversion code into service
    if datatable["rows"]:
        data["corpus"] = datatable["rows"]
        data["highlight"] = datatable["highlight"]
    else:
        data["corpus"] = None
        data["highlight"] = None
    return data
  
# Create widget 
reload_bundle()
merge_verifier = MergeVerifier({
        "init": init,
        "focus": focus
    }, mergedata)

merge_verifier.show()

component(12030cbe0, self=<kyurem.widgets.WidgetWithHistory.WidgetWithHistory object at 0x1208dd790>)

LayoutWidget(Layout(component(12030cbe0, self=<kyurem.widgets.WidgetWithHistory.WidgetWithHistory object at 0x…

In [80]:
merge_verifier.state.data.mergedata

[{'decision': 'Reject', 'entity': "A-class Driver's License", 'node': "A-class Driver's License", 'node_label': 'attribute', 'node_uuid': '0c3316198eb844778dd44b29c0003231'}, {'decision': 'Accept', 'entity': "A-class Driver's License", 'node': 'Licenses', 'node_label': 'attribute', 'node_uuid': '440c718812e945e4a77610294ebd238d'}, {'decision': 'Defer', 'entity': 'ABO Board Certification', 'node': 'ABO Board Certification', 'node_label': 'attribute', 'node_uuid': '9bd5ff0a001f4eb38dc47fa9b0983f70'}, {'decision': '---', 'entity': 'ABO Board Certification', 'node': 'Licenses', 'node_label': 'attribute', 'node_uuid': '440c718812e945e4a77610294ebd238d'}, {'decision': '---', 'entity': 'ACLS Certification', 'node': 'ACLS Certification', 'node_label': 'attribute', 'node_uuid': '0ec6aae471d34b0c8cc79eb223b120ad'}, {'decision': '---', 'entity': 'ACLS Certifications', 'node': 'Licenses', 'node_label': 'attribute', 'node_uuid': '440c718812e945e4a77610294ebd238d'}, {'decision': '---', 'entity': 'AS